#### Getting Started

Django is a high-level Python Web framework that encourages rapid development and clean, pragmatic design. Built by experienced developers, it takes care of much of the hassle of Web development, so you can focus on writing your app without needing to reinvent the wheel. It’s free and open source.

When you create an app with Django, the default settings are geared towards fast local development.  

 

Defaults are: 

- SQLite 

- Local webserver 

- Local static asset hosting 

- Built-in User model 

- Debug mode turned on 

 

We will use options that are aligned with Django community best practices, for building a proffesional website. 

This includes: <br>
- Docker for local development and deployment
- PostgreSQL
- Custom user model
- Robust user authentication flow with email
- comprehensive testing
- environment variables
- security and performance improvements and more. 

<br>

#### Contents
1. Getting Started <br>
1.1 Creating your project dir <br>
1.2 Dockerize <br>
1.3 Changing to a postgresql database <br>
1.4 Changing to a Custom User Model <br>
1.5


#### 1.1 Creating your project dir

In the command prompt create a folder in your desired destination and navigate to that folder.

Following this you install the required libaries, enter the virtual environment, and start your django project <br>

```
C:\Users\Viraj\Work>mkdir project_folder && cd project_folder

pipenv install django=2.2.3 

pipenv shell 

django-admin startproject <name_of_project> . 
```

Django starts with SQLite for your default website, however this is not scalable and efficient for a productionised website. This notebook will assume you are using a database suitable for production such as postgresql.

Django-admin startproject will kickstart your project with neccesary files needed for your project. You will slowly learn the value of each of these files as we progress through the project.

<img src="../Images/django startproject.PNG" width = "500"/>


#### 1.2 Dockerize

We will immediately create a docker container and work within here. This also prevents backtracking as your local development may not behave the same within the container. <br>

Properly configuring a local development environment remains a steep challenge despite all the other advances as we could have: 

- A different computer 

- Operating System 

- Versions of Django 

- Virtual Environment options 



In your code editor, create a dockerfile and docker-compose.yml file.

<img src="../Images/cp12dockerfile.PNG" width = "200"/>

#### Your dockerfile should contain: <br>

```
FROM python:3.7-slim

# Sets environment variables
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONBUFFERED 1

# Sets working dir, preventing need for a long code path when running commands in docker
WORKDIR /code

# Copy across pipfile and pipfile.lock, and install the dependencies
COPY Pipfile Pipfile.lock /code/
RUN pip install pipenv && pipenv install --system

# Copy across the remainding code
COPY . /code/
```

#### Your docker-compose.yml file should contain:
```
version: '3.7' 


services: 

  web: 

    build: .                                            

    command: python /code/manage.py runserver 0.0.0.0:8000 

    volumes: 

      - .:/code  

    ports: 

      - 8000:8000 

    depends_on: 

     - db 

 
 

  db: 

    image: postgres:11 

    environment: 

      - POSTGRES_PASSWORD=somepassword 
      ```

finally navigate to the folder working directory, and run the following commands:

```
docker build .
docker-compose up

```

To test this has worked correctly, navigate to http://127.0.0.1:8000/ and you should see the following screen:

DJANGO IMAGE PASTED HERE

If you recieve an error, it's likely due to the fact the project files haven't been changed to recognise postgresql as the database of choice.




#### 1.3 Changing to postgresql db

There are certain changes that must be undertaken for your django project to recognize and use the postgresql database. <br>

#### 1. The settings file should be adjusted as seen below:

DATABASES = { 

    'default': { 

        'ENGINE': 'django.db.backends.postgresql', 

        'NAME': 'postgres', 

        'USER': 'postgres', 

        'PASSWORD' : 'somepassword', #same password in environment variable 

        'HOST': 'db',  

        'PORT': 5432   

    } 

}

#### 2. pyscopg must be installed <br>
Psycopg is the most popular PostgreSQL database adapter for the Python programming language. Its main features are the complete implementation of the Python DB API 2.0 specification and the thread safety.

if your docker container is not running, enter the following command in your command prompt to start the docker container and to install the new required libary:
```
docker-compose up
docker-compose exec web pipenv --python /usr/bin/python3 install psycopg2-binary==2.8.3 

```
Please change the following section if python isn't recognised
(--python YOURFILEPATHTOPYTHON). You can also run the following line to check where python is installed on your docker container if you're unsure:
```
docker-compose exec web which python
```



#### 1.4 Customer User Model

At the start of this notebook we mentioned that the defaults are: 

- SQLite 

- Local webserver 

- Local static asset hosting 

- Built-in User model 

- Debug mode turned on 

We have already changed SQLite to postgresql, and made the required changes. This shows how we customized one of the defaults. Next we will look at creating a custom user model instead of the built in user model. It will be challenging to follow this section, for now I advise follow along and overtime this section will make your sense.

Custom user model is very difficult to switch to after the first migrate command. 

An example of why you might want the custom user model is that Django uses a username to uniquely identify a user during authentication. If you'd rather use an email address, you'll need to create a custom User model by either.

At some point you are likely to want to change the user model, therefore it's important to start with a custom user model from the start.


The four steps to create a custom user model include:
- Create a Custom User model
- Update settings.py
- Customize UserCreationForm and UserChangeForm
- Add the custom user model to admin.py

#### Step 1: Create a Custom User Model
A website is a collection of multiple "apps" to create one app. This custom user model can be created as a seperate app. Therefore the first step is to create a new directory with all the required files an app would need. Thankfully, django can create this for us without us reinventing the wheel.

```
docker-compose exec web python manage.py startapp users
```

Since we are building a model, enter the users/models.py folder and begin creating your CustomUserModel

users/models.py
from django.contrib.auth.models import AbstractUser
from django.db import models

```
class CustomUser(AbstractUser):
    pass
```

We are entirely happy with Django’s User model, but you want to add some additional profile information, you could subclass django.contrib.auth.models.AbstractUser and add your custom profile fields as seen above.

#### Step 2: Update settings.py <br>

Since we have created a new app, django must acknowledge this. Enter your_project/settings.py and add the following information to installed_apps. Since we are using a new custom user model, we must tell django this too. There isn't a default value already there as the default doesn't need to be written. However, we can add AUTH_USER_MODEL = "" and django will know to use the new user model.
```
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',

    # Local
    'users.apps.UsersConfig', #new
]

AUTH_USER_MODEL ='users.CustomUser' #new
```
Next we need to makemigrations and migrate. <br>

You should think of migrations as a version control system for your database schema. makemigrations is responsible for packaging up your model changes into individual migration files - analogous to commits - and migrate is responsible for applying those to your database.

The migration files for each app live in a “migrations” directory inside of that app, and are designed to be committed to, and distributed as part of, its codebase. You should be making them once on your development machine and then running the same migrations on your colleagues’ machines, your staging machines, and eventually your production machines.

makemigrations basically generates the SQL commands for preinstalled apps (which can be viewed in installed apps in settings.py) and your newly created apps' model which you add in installed apps.It does not execute those commands in your database file. So tables don't created after makemigrations. 

After applying makemigrations you can see those SQL commands with sqlmigrate which shows all the SQL commands which has been generated by makemigrations. 

migrate executes those SQL commands in database file. So after executing migrate all the tables of your installed apps are created in your database file. 

Migrate synchronizes the database state with the current set of models and migrations. Migrations, their relationship with apps and more are covered in depth in the migrations documentation.
https://docs.djangoproject.com/en/3.1/topics/migrations/

By default, INSTALLED_APPS contains the following apps, all of which come with Django:
<br>
django.contrib.admin – The admin site. You’ll use it shortly. <br>
django.contrib.auth – An authentication system. <br>
django.contrib.contenttypes – A framework for content types. <br>
django.contrib.sessions – A session framework.<br>
django.contrib.messages – A messaging framework. <br>
django.contrib.staticfiles – A framework for managing static files.<br>
These applications are included by default as a convenience for the common case. <br>

Some of these applications make use of at least one database table, though, so we need to create the tables in the database before we can use them. In addition we have the new custom user model. 
To do that, run the following command:
```
Docker-compose exec web python manage.py makemigrations users  

Docker-compose exec web python manage.py migrate

```

#### Step 3: Custom User Forms
In our users app we need to create a forms.py. Since a user model can also be created and edited within django admin we will need to update those built in forms too to point to custom user instead of user.

``` 
create users/forms.py

Inside this file paste the following:

from django.contrib.auth import get_user_model
from django.contrib.auth.forms import UserCreationForm, UserChangeForm

#
class CustomUserCreationForm(UserCreationForm):
'''
    A form that creates a user, with no privileges, from the given username and
    password.
    '''
    class Meta(UserCreationForm):
        model = get_user_model() # imports CustomUserModel
        fields = ('email','username',) # Fields we want to add

class CustomUserChangeForm(UserChangeForm):
''' 
A form used in the admin interface to change a user’s information and permissions.
'''
    class Meta(UserChangeForm):
        model = get_user_model()
        fields = ('email','username',) #
```
